## Setups and imports

**Imports**

In [24]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy
import seaborn as sns

from scipy.stats import zscore
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

**Dataset loading and opening**

In [25]:
# Dataset
data = pd.read_csv('data.csv')
data.head()

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,Q91E,country,dateload,introelapse,testelapse,surveyelapse,gender,engnat,age,IE
0,5,51,7107,3,91,2522,1,56,6180,2,...,4609,US,2019-02-20 17:35:52,1,461,16,2,1,23,3
1,5,39,6354,5,13,3092,1,12,5243,5,...,10409,AU,2019-02-20 17:46:32,21,467,15,1,2,25,2
2,3,17,5397,4,35,2747,5,40,5262,3,...,2691,BR,2019-02-20 18:10:24,56,306,17,1,2,19,1
3,5,41,3055,2,14,3348,1,13,5141,1,...,3697,CZ,2019-02-20 18:16:21,2,287,14,1,1,23,1
4,1,76,2542,2,54,1878,1,15,5637,1,...,1662,CA,2019-02-20 18:21:49,2,325,12,1,1,18,2


## Data quality handling

**Missing Values**

Assess the number of missing values:

In [26]:
# Check for missing values
columns = data.columns
for name in columns:
    print(f"{name}: {data[name].isna().unique()}")

Q1A: [False]
Q1I: [False]
Q1E: [False]
Q2A: [False]
Q2I: [False]
Q2E: [False]
Q3A: [False]
Q3I: [False]
Q3E: [False]
Q4A: [False]
Q4I: [False]
Q4E: [False]
Q5A: [False]
Q5I: [False]
Q5E: [False]
Q6A: [False]
Q6I: [False]
Q6E: [False]
Q7A: [False]
Q7I: [False]
Q7E: [False]
Q8A: [False]
Q8I: [False]
Q8E: [False]
Q9A: [False]
Q9I: [False]
Q9E: [False]
Q10A: [False]
Q10I: [False]
Q10E: [False]
Q11A: [False]
Q11I: [False]
Q11E: [False]
Q12A: [False]
Q12I: [False]
Q12E: [False]
Q13A: [False]
Q13I: [False]
Q13E: [False]
Q14A: [False]
Q14I: [False]
Q14E: [False]
Q15A: [False]
Q15I: [False]
Q15E: [False]
Q16A: [False]
Q16I: [False]
Q16E: [False]
Q17A: [False]
Q17I: [False]
Q17E: [False]
Q18A: [False]
Q18I: [False]
Q18E: [False]
Q19A: [False]
Q19I: [False]
Q19E: [False]
Q20A: [False]
Q20I: [False]
Q20E: [False]
Q21A: [False]
Q21I: [False]
Q21E: [False]
Q22A: [False]
Q22I: [False]
Q22E: [False]
Q23A: [False]
Q23I: [False]
Q23E: [False]
Q24A: [False]
Q24I: [False]
Q24E: [False]
Q25A: [False]
Q25I:

In [27]:
print(f"Country att/>ribute null sum: {data['country'].isna().sum()}")
print(f"Unique values in country column: {data['country'].unique()}")

Country att/>ribute null sum: 2
Unique values in country column: ['US' 'AU' 'BR' 'CZ' 'CA' 'MX' 'IN' 'GB' 'PH' 'NONE' 'DE' 'PY' 'FI' 'GR'
 'SG' 'PL' 'OM' 'NZ' 'MY' 'KR' 'SK' 'RO' 'ID' 'PE' 'IT' 'CY' 'UA' 'LT'
 'KY' 'SE' 'RS' 'NL' 'FR' 'TT' 'RU' 'HR' 'IE' 'ZA' 'AL' 'HU' 'BE' 'PT'
 'CO' 'HN' 'MN' 'GH' 'JP' 'AE' 'BA' 'CR' 'LK' 'NO' 'VN' 'AT' 'CL' 'JM'
 'CH' 'DK' 'TR' 'ES' 'AR' 'BN' 'NG' 'TH' 'CN' 'EC' 'GE' 'TW' 'SI' 'HK'
 'BG' 'UY' 'MU' 'LV' 'PK' 'AG' 'GU' 'KE' 'EG' 'IS' 'IL' 'KZ' 'DZ' 'BY'
 'SA' 'SC' 'ME' 'MA' 'MT' 'BS' 'MK' 'ZM' 'TZ' 'SV' 'JO' 'BH' 'LB' 'VE'
 'AX' 'AM' 'ZW' nan 'LU' 'EE' 'MD' 'NP' 'SO' 'TN' 'KW' 'QA' 'PA' 'BB' 'JE'
 'GI' 'PR' 'GT' 'AZ' 'TC' 'KN' 'SY' 'DO' 'RW']


In [28]:
print(Counter(data['country']))

Counter({'US': 3417, 'GB': 517, 'CA': 448, 'AU': 323, 'DE': 193, 'NONE': 185, 'IN': 150, 'PH': 127, 'ID': 109, 'BR': 91, 'PL': 91, 'RO': 77, 'MY': 76, 'NL': 74, 'NZ': 70, 'FR': 68, 'SE': 65, 'FI': 63, 'SG': 62, 'IT': 58, 'RU': 48, 'PT': 41, 'ES': 38, 'NO': 35, 'IE': 34, 'MX': 32, 'ZA': 31, 'GR': 27, 'CZ': 26, 'DK': 26, 'TR': 26, 'RS': 25, 'BE': 24, 'AR': 24, 'HR': 23, 'CH': 22, 'HU': 21, 'LT': 19, 'NG': 19, 'PK': 17, 'JP': 16, 'AT': 16, 'KE': 16, 'UA': 15, 'HK': 15, 'TH': 14, 'VN': 13, 'SI': 13, 'BG': 13, 'CN': 10, 'IL': 10, 'SK': 9, 'AE': 9, 'KR': 8, 'CO': 8, 'CL': 8, 'BY': 8, 'LB': 8, 'PY': 7, 'GE': 7, 'MA': 6, 'TT': 5, 'AL': 5, 'BA': 5, 'CR': 5, 'JM': 5, 'TW': 5, 'DZ': 5, 'SA': 5, 'PE': 4, 'EC': 4, 'MK': 4, 'EE': 4, 'MN': 3, 'LV': 3, 'EG': 3, 'ME': 3, 'BS': 3, 'VE': 3, 'AM': 3, 'MD': 3, 'NP': 3, 'CY': 2, 'KY': 2, 'HN': 2, 'LK': 2, 'BN': 2, 'UY': 2, 'MU': 2, 'IS': 2, 'MT': 2, 'ZM': 2, nan: 2, 'QA': 2, 'PA': 2, 'GT': 2, 'OM': 1, 'GH': 1, 'AG': 1, 'GU': 1, 'KZ': 1, 'SC': 1, 'TZ': 1, 'S

The total number of missing values in 187, which is around 2.6% of the total dataset. We conclude that dropping the missing values will not have significant impact.

In [29]:
data = data.dropna()

for country in data['country']:
    idx = data[data['country']=='NONE'].index
    data = data.drop(idx)

In [30]:
print(f"Country att/>ribute null sum: {data['country'].isna().sum()}")
print(f"Unique values in country column: {data['country'].unique()}")

Country att/>ribute null sum: 0
Unique values in country column: ['US' 'AU' 'BR' 'CZ' 'CA' 'MX' 'IN' 'GB' 'PH' 'DE' 'PY' 'FI' 'GR' 'SG'
 'PL' 'OM' 'NZ' 'MY' 'KR' 'SK' 'RO' 'ID' 'PE' 'IT' 'CY' 'UA' 'LT' 'KY'
 'SE' 'RS' 'NL' 'FR' 'TT' 'RU' 'HR' 'IE' 'ZA' 'AL' 'HU' 'BE' 'PT' 'CO'
 'HN' 'MN' 'GH' 'JP' 'AE' 'BA' 'CR' 'LK' 'NO' 'VN' 'AT' 'CL' 'JM' 'CH'
 'DK' 'TR' 'ES' 'AR' 'BN' 'NG' 'TH' 'CN' 'EC' 'GE' 'TW' 'SI' 'HK' 'BG'
 'UY' 'MU' 'LV' 'PK' 'AG' 'GU' 'KE' 'EG' 'IS' 'IL' 'KZ' 'DZ' 'BY' 'SA'
 'SC' 'ME' 'MA' 'MT' 'BS' 'MK' 'ZM' 'TZ' 'SV' 'JO' 'BH' 'LB' 'VE' 'AX'
 'AM' 'ZW' 'LU' 'EE' 'MD' 'NP' 'SO' 'TN' 'KW' 'QA' 'PA' 'BB' 'JE' 'GI'
 'PR' 'GT' 'AZ' 'TC' 'KN' 'SY' 'DO' 'RW']


In [31]:
print(Counter(data['country']))

Counter({'US': 3417, 'GB': 517, 'CA': 448, 'AU': 323, 'DE': 193, 'IN': 150, 'PH': 127, 'ID': 109, 'BR': 91, 'PL': 91, 'RO': 77, 'MY': 76, 'NL': 74, 'NZ': 70, 'FR': 68, 'SE': 65, 'FI': 63, 'SG': 62, 'IT': 58, 'RU': 48, 'PT': 41, 'ES': 38, 'NO': 35, 'IE': 34, 'MX': 32, 'ZA': 31, 'GR': 27, 'CZ': 26, 'DK': 26, 'TR': 26, 'RS': 25, 'BE': 24, 'AR': 24, 'HR': 23, 'CH': 22, 'HU': 21, 'LT': 19, 'NG': 19, 'PK': 17, 'JP': 16, 'AT': 16, 'KE': 16, 'UA': 15, 'HK': 15, 'TH': 14, 'VN': 13, 'SI': 13, 'BG': 13, 'CN': 10, 'IL': 10, 'SK': 9, 'AE': 9, 'KR': 8, 'CO': 8, 'CL': 8, 'BY': 8, 'LB': 8, 'PY': 7, 'GE': 7, 'MA': 6, 'TT': 5, 'AL': 5, 'BA': 5, 'CR': 5, 'JM': 5, 'TW': 5, 'DZ': 5, 'SA': 5, 'PE': 4, 'EC': 4, 'MK': 4, 'EE': 4, 'MN': 3, 'LV': 3, 'EG': 3, 'ME': 3, 'BS': 3, 'VE': 3, 'AM': 3, 'MD': 3, 'NP': 3, 'CY': 2, 'KY': 2, 'HN': 2, 'LK': 2, 'BN': 2, 'UY': 2, 'MU': 2, 'IS': 2, 'MT': 2, 'ZM': 2, 'QA': 2, 'PA': 2, 'GT': 2, 'OM': 1, 'GH': 1, 'AG': 1, 'GU': 1, 'KZ': 1, 'SC': 1, 'TZ': 1, 'SV': 1, 'JO': 1, 'BH':

**Data format**

We will now ensure that the data types match the significance of the columns. We will also be ensuring that any non-numerical values are changed to numerical ones.

In [32]:
for column_name in columns:
    print(f"{column_name} data type: {data[column_name].dtype}")

Q1A data type: int64
Q1I data type: int64
Q1E data type: int64
Q2A data type: int64
Q2I data type: int64
Q2E data type: int64
Q3A data type: int64
Q3I data type: int64
Q3E data type: int64
Q4A data type: int64
Q4I data type: int64
Q4E data type: int64
Q5A data type: int64
Q5I data type: int64
Q5E data type: int64
Q6A data type: int64
Q6I data type: int64
Q6E data type: int64
Q7A data type: int64
Q7I data type: int64
Q7E data type: int64
Q8A data type: int64
Q8I data type: int64
Q8E data type: int64
Q9A data type: int64
Q9I data type: int64
Q9E data type: int64
Q10A data type: int64
Q10I data type: int64
Q10E data type: int64
Q11A data type: int64
Q11I data type: int64
Q11E data type: int64
Q12A data type: int64
Q12I data type: int64
Q12E data type: int64
Q13A data type: int64
Q13I data type: int64
Q13E data type: int64
Q14A data type: int64
Q14I data type: int64
Q14E data type: int64
Q15A data type: int64
Q15I data type: int64
Q15E data type: int64
Q16A data type: int64
Q16I data type:

In [36]:
data['country'] = data['country'].astype(str)
print(f"{data['country'].dtype}")

KeyError: 'country'

In [ ]:
d